In [1]:
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet import preprocess_input
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
training_set = train_datagen.flow_from_directory('../../Covid19-dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 251 images belonging to 3 classes.


In [11]:
test_set = test_datagen.flow_from_directory('../../Covid19-dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 66 images belonging to 3 classes.


In [12]:
model = load_model('../../models/covid/covid_vgg1.h5')

In [6]:
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

3/3 [==============================] - 26s 6s/step - loss: 0.0872 - accuracy: 0.9697
Test Loss: 0.08715351670980453, Test Accuracy: 0.9696969985961914


In [8]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [23]:
def preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

def visualize_saliency_map(model, image_path, output_image_path, output_image_name, category):
    img = preprocess_image(image_path)
    img_tensor = tf.convert_to_tensor(img)  # Convert the NumPy array to a TensorFlow tensor
    
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        predictions = model(img_tensor)
        loss = tf.keras.backend.max(predictions, axis=1)
    
    gradients = tape.gradient(loss, img_tensor)
    saliency_map = np.max(np.abs(gradients.numpy()), axis=-1)[0]

    # Normalize the saliency map
    saliency_map = (saliency_map - np.min(saliency_map)) / (np.max(saliency_map) - np.min(saliency_map))

    # Visualize the original image and the saliency map
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img[0])
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(saliency_map, cmap='gray')
    plt.title(f'Saliency Map ({category})')
    plt.axis('off')

    saliency_map_path = os.path.join(output_image_path, output_image_name)

    plt.subplot(1, 2, 2)
    plt.savefig(saliency_map_path)

    plt.show()

In [24]:
input_directory = '../../Covid19-dataset/test/'
output_directory = '../../Output_Images/saliency_map/'
images_to_process = 5

In [25]:
for i, category in enumerate(os.listdir(input_directory)):
    num_images_processed = 0
    image_directory = os.path.join(output_directory, category)
    os.makedirs(image_directory, exist_ok=True)
    for j, filename in enumerate(os.listdir(os.path.join(input_directory, category))):
        visualize_saliency_map(model, os.path.join(input_directory, category, filename), image_directory, f'{category}_{j+1}_saliency_map.png', category)
        num_images_processed += 1
        
        # Check if we have processed the desired number of images
        if num_images_processed == images_to_process:
            break

TypeError: visualize_saliency_map() missing 1 required positional argument: 'category'